# setup

## load repo, libs, and functions

In [ ]:
%%capture
# ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`
# must do this at start of nb
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
# get the data and functions
!git clone https://github.com/nrimsky/CAA.git

Cloning into 'CAA'...
remote: Enumerating objects: 17180, done.
remote: Counting objects: 100% (2168/2168), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 17180 (delta 2115), reused 2052 (delta 2052), pack-reused 15012
Receiving objects: 100% (17180/17180), 1.16 GiB | 50.33 MiB/s, done.
Resolving deltas: 100% (10803/10803), done.
Updating files: 100% (5370/5370), done.


In [ ]:
import sys
sys.path.append('/content/CAA')

In [ ]:
# see requirements.txt
!pip install python-dotenv==1.0.0

In [ ]:
# import generate_vectors
from generate_vectors import *

In [ ]:
import json
import torch as t
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from tqdm import tqdm
import os
from dotenv import load_dotenv
from llama_wrapper import LlamaWrapper
import argparse
from typing import List
from utils.tokenize import tokenize_llama_base, tokenize_llama_chat
from behaviors import (
    get_vector_dir,
    get_activations_dir,
    get_ab_data_path,
    get_vector_path,
    get_activations_path,
    ALL_BEHAVIORS
)

load_dotenv()

# HUGGINGFACE_TOKEN = os.getenv("HF_TOKEN")

False

In [ ]:
import torch.nn.functional as F

# hf key

Do this to avoid gated error (Llama 2 needs permission from meta to grant your token key access)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # place token here

In [ ]:
# os.environ['HUGGINGFACE_CO_API_KEY'] = '' # token here

# Load Model

## gpt2

In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# model_name = "gpt2-xl"
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

## Llama-7b

In [ ]:
use_base_model = True
model_size = '7b'
HUGGINGFACE_TOKEN = os.getenv("HF_TOKEN")

model = LlamaWrapper(
        HUGGINGFACE_TOKEN, size=model_size, use_chat=not use_base_model
    )
# model.set_save_internal_decodings(False)
# model.reset_all()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

If get: OutOfMemoryError: CUDA out of memory.

Try: low_cpu_mem_usage. Load without wrapper first, then instead of using wrapper's methods, paste the methods here to analyze their internals

In [ ]:
# from transformers import LlamaForCausalLM, LlamaTokenizer
# LLAMA_2_7B_CHAT_PATH = "meta-llama/Llama-2-7b-chat-hf"

# tokenizer = LlamaTokenizer.from_pretrained(LLAMA_2_7B_CHAT_PATH)
# hf_model = LlamaForCausalLM.from_pretrained(LLAMA_2_7B_CHAT_PATH, low_cpu_mem_usage=True)

# Generate by 2 prompts

In [ ]:
# do this b/c anger is one token, calm is 2, so this pads anger with 50256
# tokens = model.to_tokens(batch_input) # for Transformerlens

p_prompt = ["first"]
n_prompt = ["one"]

# , return_tensors="pt") is returns type. "pt" stands for PyTorch tensors
# model only takes pytorch tensors in, not list
p_tokens = model.tokenizer(p_prompt, return_tensors="pt")['input_ids']
p_tokens = p_tokens.to(model.device)
n_tokens = model.tokenizer(n_prompt, return_tensors="pt")['input_ids']
n_tokens = n_tokens.to(model.device)

print(p_tokens)
print(n_tokens)

tensor([[  1, 937]], device='cuda:0')
tensor([[  1, 697]], device='cuda:0')


In [ ]:
layers = [15]

model.reset_all()
model.get_logits(p_tokens)
for layer in layers:
    p_activations = model.get_last_activations(layer)
    p_activations = p_activations[0, -1, :].detach().cpu()
model.reset_all()
model.get_logits(n_tokens)
for layer in layers:
    n_activations = model.get_last_activations(layer)
    n_activations = n_activations[0, -1, :].detach().cpu()

In [ ]:
for layer in layers:
    vec = (p_activations - n_activations)

In [ ]:
vec.shape

torch.Size([4096])

# Steer

In [ ]:
layer = 15
multiplier = 0
model.set_save_internal_decodings(True)
model.reset_all()
model.set_add_activations(layer, multiplier * vec.cuda())
prompt = ["second"]
tokens = model.tokenizer(prompt, return_tensors="pt")['input_ids']
tokens = tokens.to(model.device)

output = model.model(tokens)
print(len(output))
print(output[0].shape)
print(len(output[1]))

2
torch.Size([1, 2, 32000])
32


In [ ]:
output[0][0, -1].shape

torch.Size([32000])

In [ ]:
next_token = output[0][0, -1].argmax(dim=-1)
model.tokenizer.decode(next_token)
# next_char = model.model.to_string(next_token)
# next_char

'hand'

## ranks to numwords

In [ ]:
layer = 15
multiplier = 0
model.set_save_internal_decodings(True)
model.reset_all()
model.set_add_activations(layer, multiplier * vec.cuda())
prompt = ["two"]
tokens = model.tokenizer(prompt, return_tensors="pt")['input_ids']
tokens = tokens.to(model.device)

output = model.model(tokens)
next_token = output[0][0, -1].argmax(dim=-1)
model.tokenizer.decode(next_token)

'-'

# loop thru layers to find steer vecs

# first to one

In [ ]:
p_prompt = ["first"]
n_prompt = ["one"]

# , return_tensors="pt") is returns type. "pt" stands for PyTorch tensors
# model only takes pytorch tensors in, not list
p_tokens = model.tokenizer(p_prompt, return_tensors="pt")['input_ids']
p_tokens = p_tokens.to(model.device)
n_tokens = model.tokenizer(n_prompt, return_tensors="pt")['input_ids']
n_tokens = n_tokens.to(model.device)

print(p_tokens)
print(n_tokens)

tensor([[  1, 937]], device='cuda:0')
tensor([[  1, 697]], device='cuda:0')


In [ ]:
layers = [i for i in range(0, 31)]
pos_activations = dict([(layer, []) for layer in layers])
neg_activations = dict([(layer, []) for layer in layers])
vec_layer_dict = dict([(layer, []) for layer in layers])

model.reset_all()
model.get_logits(p_tokens)
for layer in layers:
    p_activations = model.get_last_activations(layer)
    p_activations = p_activations[0, -1, :].detach().cpu()
    pos_activations[layer] = p_activations
model.reset_all()
model.get_logits(n_tokens)
for layer in layers:
    n_activations = model.get_last_activations(layer)
    n_activations = n_activations[0, -1, :].detach().cpu()
    neg_activations[layer] = n_activations

for layer in layers:
    p_activations = pos_activations[layer]
    n_activations = neg_activations[layer]
    vec = (p_activations - n_activations)
    vec_layer_dict[layer] = vec

In [ ]:
# prompt = ["first second third "]
prompt = ["first"]
tokens = model.tokenizer(prompt, return_tensors="pt")['input_ids']
tokens = tokens.to(model.device)

for layer in layers:
    model.set_save_internal_decodings(True)
    model.reset_all()

    multiplier = 5
    vec = vec_layer_dict[layer]
    model.set_add_activations(layer, multiplier * vec.cuda())

    output = model.model(tokens)
    next_token = output[0][0, -1].argmax(dim=-1)
    next_char = model.tokenizer.decode(next_token)
    print(layer, next_char)

0 -
1 -
2 -
3 -
4 second
5 second
6 -
7 
8 -
9 second
10 time
11 ,
12 impress
13 impress
14 impress
15 aid
16 aid
17 aid
18 aid
19 aid
20 aid
21 aid
22 aid
23 enberg
24 aid
25 aid
26 published
27 published
28 rade
29 ly
30 ly


In [ ]:
prompt = ["first second third "]
tokens = model.tokenizer(prompt, return_tensors="pt")['input_ids']
tokens = tokens.to(model.device)

for layer in layers:
    print(layer)

    # model.set_save_internal_decodings(True)
    model.reset_all()
    print("Before steering:")

    output_tokens = model.model.generate(tokens, max_length= 10)
    generated_text = model.tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(generated_text)

    model.reset_all()
    print("After steering:")

    multiplier = 5
    vec = vec_layer_dict[layer]
    model.set_add_activations(layer, multiplier * vec.cuda())

    output_tokens = model.model.generate(tokens, max_length= 10)
    generated_text = model.tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(generated_text)

0
Before steering:
first second third 1000th
After steering:
first second third  first last second second second
1
Before steering:
first second third 4th 5th
After steering:
first second third  first second first second first
2
Before steering:
first second third 4th 5th
After steering:
first second third .
 first first-
3
Before steering:
first second third 4th 5th
After steering:
first second third  edition edition edition second edition
4
Before steering:
first second third 4th 5th
After steering:
first second third  first First first first first
5
Before steering:
first second third 4th 5th
After steering:
first second third  example examples examples. First
6
Before steering:
first second third 4th 5th
After steering:
first second third  The D. First.
7
Before steering:
first second third 4th 5th
After steering:
first second third      
8
Before steering:
first second third 1st 2nd
After steering:
first second third  child second
First child
9
Before steering:
first second third 

# love to hate

In [ ]:
p_prompt = ["love"]
n_prompt = ["hate"]

# , return_tensors="pt") is returns type. "pt" stands for PyTorch tensors
# model only takes pytorch tensors in, not list
p_tokens = model.tokenizer(p_prompt, return_tensors="pt")['input_ids']
p_tokens = p_tokens.to(model.device)
n_tokens = model.tokenizer(n_prompt, return_tensors="pt")['input_ids']
n_tokens = n_tokens.to(model.device)

print(p_tokens)
print(n_tokens)

tensor([[   1, 5360]], device='cuda:0')
tensor([[    1, 26277]], device='cuda:0')


In [ ]:
layers = [i for i in range(0, 31)]
pos_activations = dict([(layer, []) for layer in layers])
neg_activations = dict([(layer, []) for layer in layers])
vec_layer_dict = dict([(layer, []) for layer in layers])

model.reset_all()
model.get_logits(p_tokens)
for layer in layers:
    p_activations = model.get_last_activations(layer)
    p_activations = p_activations[0, -1, :].detach().cpu()
    pos_activations[layer] = p_activations
model.reset_all()
model.get_logits(n_tokens)
for layer in layers:
    n_activations = model.get_last_activations(layer)
    n_activations = n_activations[0, -1, :].detach().cpu()
    neg_activations[layer] = n_activations

for layer in layers:
    p_activations = pos_activations[layer]
    n_activations = neg_activations[layer]
    vec = (p_activations - n_activations)
    vec_layer_dict[layer] = vec

In [ ]:
prompt = ["I hate you because"]
# prompt = ["love"]
# prompt = ["hate"]
tokens = model.tokenizer(prompt, return_tensors="pt")['input_ids']
tokens = tokens.to(model.device)

for layer in layers:
    # model.set_save_internal_decodings(True)
    model.reset_all()

    multiplier = 5
    vec = vec_layer_dict[layer]
    model.set_add_activations(layer, multiplier * vec.cuda())

    output = model.model(tokens)
    next_token = output[0][0, -1].argmax(dim=-1)
    next_char = model.tokenizer.decode(next_token)
    print(layer, next_char)

0 love
1 love
2 and
3 and
4 and
5 and
6 ,
7 ,
8 ,
9 ,
10 ,
11 and
12 .
13 I
14 ,
15 of
16 of
17 rose
18 interests
19 aff
20 abund
21 ee
22 interests
23 interests
24 interests
25 interests
26 interests
27 Bug
28 Bug
29 Bug
30 Tar


In [ ]:
model.reset_all()
output_tokens = model.model.generate(tokens, max_length=10)
generated_text = model.tokenizer.decode(output_tokens[0], skip_special_tokens=True)
generated_text

'I hate you because you are beautiful.\n'

In [ ]:
prompt = ["I hate you because "]
tokens = model.tokenizer(prompt, return_tensors="pt")['input_ids']
tokens = tokens.to(model.device)

for layer in layers:
    print(layer)

    # model.set_save_internal_decodings(True)
    model.reset_all()
    print("Before steering:")

    output_tokens = model.model.generate(tokens, max_length= 10)
    generated_text = model.tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(generated_text)

    model.reset_all()
    print("After steering:")

    multiplier = 5
    vec = vec_layer_dict[layer]
    model.set_add_activations(layer, multiplier * vec.cuda())

    output_tokens = model.model.generate(tokens, max_length= 10)
    generated_text = model.tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(generated_text)

0
Before steering:
I hate you because 15 years ago
After steering:
I hate you because  love love love love
1
Before steering:
I hate you because 2011
After steering:
I hate you because  love love love love
2
Before steering:
I hate you because 1000
After steering:
I hate you because . I love.
3
Before steering:
I hate you because 2011
After steering:
I hate you because  and.
.
4
Before steering:
I hate you because 2011
After steering:
I hate you because  and and and and
5
Before steering:
I hate you because 1997
After steering:
I hate you because  is and how,
6
Before steering:
I hate you because 1. You are
After steering:
I hate you because  and and love,
7
Before steering:
I hate you because 1. I’
After steering:
I hate you because .
, and
8
Before steering:
I hate you because 1) I'
After steering:
I hate you because , and the it
9
Before steering:
I hate you because 100 years
After steering:
I hate you because .

,
10
Before steering:
I hate you because 1980
After steering:
I hate y